# Agentic RAG with PostgreSQL + ChromaDB

This notebook demonstrates an advanced Agentic RAG system that combines:
- **PostgreSQL**: Structured game metadata and critic reviews
- **ChromaDB**: Vector embeddings for semantic search
- **LlamaIndex**: Intelligent agent with multi-step reasoning

## What's Different from the Original:
- ✅ Uses PostgreSQL for fast metadata queries
- ✅ Leverages the `games_with_critics` view for rich data
- ✅ Maintains ChromaDB for vector search
- ✅ Better performance and data consistency
- ✅ Structured data with proper relationships

## Database Schema:
- **games**: 188,267 games with metadata
- **critics**: 609,000 critic reviews
- **games_with_critics**: View combining games + critics
- **ChromaDB**: Vector embeddings for semantic search


In [ ]:
# Install required packages
# %pip install llama-index llama-index-llms-ollama llama-index-embeddings-huggingface
# %pip install psycopg2-binary

print("✅ Packages ready!")

✅ Packages ready!


In [1]:
# Import all required libraries
import psycopg2
import psycopg2.extras
import chromadb
from chromadb.config import Settings as ChromaSettings, DEFAULT_TENANT, DEFAULT_DATABASE
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch
import re
import json
from typing import List, Dict, Any

from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core import Settings

print("✅ All imports successful!")

✅ All imports successful!


In [2]:
# Database and model configuration
DB_CONFIG = {
    'host': 'localhost',
    'database': 'gamequest',
    'user': 'postgres',
    'password': 'ST5780@BCsp'
}

# Initialize PostgreSQL connection
def get_db_connection():
    return psycopg2.connect(**DB_CONFIG)

# Test database connection
try:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM games;")
    games_count = cursor.fetchone()[0]
    cursor.execute("SELECT COUNT(*) FROM critics;")
    critics_count = cursor.fetchone()[0]
    cursor.close()
    conn.close()
    print(f"✅ PostgreSQL connected! Games: {games_count:,}, Critics: {critics_count:,}")
except Exception as e:
    print(f"❌ Database connection failed: {e}")

# Initialize ChromaDB
client = chromadb.PersistentClient(
    path="../chroma_db",
    settings=ChromaSettings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# Get ChromaDB collections
desc_collection = client.get_or_create_collection("desc_embeddings")
critics_collection = client.get_or_create_collection("critics_embeddings")
cover_collection = client.get_or_create_collection("cover_embeddings")
screenshot_collection = client.get_or_create_collection("screenshot_embeddings")

print("✅ ChromaDB collections loaded!")


✅ PostgreSQL connected! Games: 188,267, Critics: 609,000
✅ ChromaDB collections loaded!


In [3]:
# Load ML models
print("🔄 Loading ML models...")

desc_encoder = SentenceTransformer("../models/all-MiniLM-L6-v2")
reranker = CrossEncoder("../models/bge-reranker-base-crossencoder")
processor = AutoImageProcessor.from_pretrained("../models/dinov2-base")
viz_encoder = AutoModel.from_pretrained("../models/dinov2-base")

print("✅ All models loaded successfully!")


🔄 Loading ML models...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ All models loaded successfully!


In [4]:
# Utility functions for PostgreSQL integration
def clean_html(text):
    """Remove HTML tags from text"""
    if not text:
        return ''
    return re.sub('<[^<]+?>', '', str(text)).strip()

def get_game_from_db(game_id: int) -> Dict[str, Any]:
    """Get game data from PostgreSQL using the games_with_critics view"""
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    
    cursor.execute("""
        SELECT id, title, description, release_date, moby_score, moby_url,
               platforms, genres, developers, publishers, cover_path, 
               screenshot_paths, all_critics
        FROM games_with_critics 
        WHERE id = %s
    """, (game_id,))
    
    result = cursor.fetchone()
    cursor.close()
    conn.close()
    
    if result:
        # Convert to regular dict and clean HTML
        game_data = dict(result)
        game_data['description'] = clean_html(game_data.get('description', ''))
        
        # Combine critic citations
        critic_citations = game_data.get('critic_citations', [])
        if critic_citations:
            clean_citations = [clean_html(citation) for citation in critic_citations if citation]
            game_data['critic'] = ' | '.join(clean_citations)
        else:
            game_data['critic'] = ''
        
        return game_data
    
    return None

def get_games_from_db(game_ids: List[int]) -> List[Dict[str, Any]]:
    """Get multiple games from PostgreSQL"""
    if not game_ids:
        return []
    
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    
    # Create placeholders for the IN clause
    placeholders = ','.join(['%s'] * len(game_ids))
    
    cursor.execute(f"""
        SELECT id, title, description, release_date, moby_score, moby_url,
               platforms, genres, developers, publishers, cover_path, 
               screenshot_paths, all_critics
        FROM games_with_critics 
        WHERE id IN ({placeholders})
    """, game_ids)
    
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    
    games = []
    for result in results:
        game_data = dict(result)
        game_data['description'] = clean_html(game_data.get('description', ''))
        
        # Combine critic citations (all_critics is a single string with pipe separators)
        all_critics = game_data.get('all_critics', '')
        if all_critics:
            # Split by pipe and clean each citation
            citations = all_critics.split('|')
            clean_citations = [clean_html(citation.strip()) for citation in citations if citation.strip()]
            game_data['critic'] = ' | '.join(clean_citations)
        else:
            game_data['critic'] = ''
        
        games.append(game_data)
    
    return games

def format_game_result(game_data: Dict[str, Any], score: float = None) -> str:
    """Format a game result for display"""
    result = f"🎮 {game_data['title']}"
    if score is not None:
        result += f" (Score: {score:.2f})"
    
    if game_data.get('description'):
        desc = game_data['description'][:200] + "..." if len(game_data['description']) > 200 else game_data['description']
        result += f"\n📝 {desc}"
    
    if game_data.get('critic'):
        critic = game_data['critic'][:100] + "..." if len(game_data['critic']) > 100 else game_data['critic']
        result += f"\n⭐ Critics: {critic}"
    
    if game_data.get('platforms'):
        platforms = ', '.join(game_data['platforms'][:3])  # Show first 3 platforms
        result += f"\n🎯 Platforms: {platforms}"
    
    return result

print("✅ Utility functions ready!")


✅ Utility functions ready!


## Enhanced Agent Tools with PostgreSQL Integration

Now we'll create agent tools that use PostgreSQL for fast metadata retrieval while keeping ChromaDB for vector search.

In [5]:
# Tool 1: Enhanced Text Search with PostgreSQL
def text_search_tool(query: str, num_results: int = 10) -> str:
    """
    Search for games using text descriptions and reviews with PostgreSQL integration.
    
    Args:
        query: Text description of what you're looking for
        num_results: Number of results to return (default: 10)
    
    Returns:
        Formatted string with game recommendations
    """
    print(f"🔍 Searching for: '{query}'")
    
    # Get query embedding
    query_embedding = desc_encoder.encode(query)
    
    # Search descriptions in ChromaDB
    desc_results = desc_collection.query(
        query_embeddings=[query_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    # Search critics in ChromaDB
    critics_results = critics_collection.query(
        query_embeddings=[query_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    # Collect unique game IDs
    game_ids = set()
    for meta_list in [desc_results["metadatas"][0], critics_results["metadatas"][0]]:
        for meta in meta_list:
            game_ids.add(int(meta["game_id"]))
    
    # Get game data from PostgreSQL (much faster than loading JSONL)
    games = get_games_from_db(list(game_ids))
    
    # Create candidates for reranking
    candidates = []
    for game in games:
        candidates.append({
            "game_id": game["id"],
            "title": game["title"],
            "description": game["description"],
            "critic": game["critic"],
            "platforms": game.get("platforms", []),
            "genres": game.get("genres", [])
        })
    
    # Rerank using cross-encoder
    pairs = []
    for c in candidates:
        candidate_text = f"{c['title']}. {c['description']} Critics: {c['critic']}"
        pairs.append([query, candidate_text])
    
    if pairs:
        scores = reranker.predict(pairs, batch_size=16)
        reranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    else:
        reranked = []
    
    # Format results
    result = f"Found {len(reranked)} games matching '{query}':\n\n"
    for i, (c, score) in enumerate(reranked[:num_results], 1):
        result += f"{i}. {format_game_result(c, score)}\n\n"
    
    return result

# Test the enhanced tool
test_result = text_search_tool("survival games with crafting", 3)
print("🧪 Testing enhanced text search tool:")
print(test_result)


🔍 Searching for: 'survival games with crafting'
🧪 Testing enhanced text search tool:
Found 2 games matching 'survival games with crafting':

1. 🎮 Craft the World (Score: 0.79)
📝 Craft The World is a 2D sandbox strategy game, combining the gameplay elements of Terraria and Dwarf Fortress into its own fantasy world, where a group of dwarves attempts to survive against onslaught...
⭐ Critics: Craft the World is a solid game.  Resources are plentiful and creatures are sporadic enough to reall...
🎯 Platforms: Windows, Macintosh, iPhone

2. 🎮 Craft (Score: 0.57)
📝 Craft is a puzzle game where the player is presented with a number of tiles where each piece has a part of a circuit. The objective is to create a complete circuit from an entry point to an exit so th...
🎯 Platforms: SAM Coupé




In [6]:
# Tool 2: Enhanced Image Search with PostgreSQL
def image_search_tool(image_path: str, num_results: int = 10) -> str:
    """
    Search for games using image similarity with PostgreSQL integration.
    
    Args:
        image_path: Path to the image file
        num_results: Number of results to return (default: 10)
    
    Returns:
        Formatted string with visually similar games
    """
    print(f"🖼️ Searching for games similar to: {image_path}")
    
    # Load and process image
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(viz_encoder.device) for k, v in inputs.items()}
    
    # Get image embedding
    with torch.no_grad():
        outputs = viz_encoder(**inputs)
        image_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    
    # Search both cover and screenshot collections
    cover_results = cover_collection.query(
        query_embeddings=[image_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    screenshot_results = screenshot_collection.query(
        query_embeddings=[image_embedding],
        n_results=num_results//2,
        include=["metadatas", "distances"]
    )
    
    # Collect unique game IDs
    game_ids = set()
    cover_scores = {}
    screenshot_scores = {}
    
    for meta, dist in zip(cover_results["metadatas"][0], cover_results["distances"][0]):
        game_id = int(meta["game_id"])
        game_ids.add(game_id)
        cover_scores[game_id] = 1 - dist  # Convert distance to similarity
    
    for meta, dist in zip(screenshot_results["metadatas"][0], screenshot_results["distances"][0]):
        game_id = int(meta["game_id"])
        game_ids.add(game_id)
        screenshot_scores[game_id] = 1 - dist
    
    # Get game data from PostgreSQL
    games = get_games_from_db(list(game_ids))
    
    # Create candidates with combined scores
    candidates = []
    for game in games:
        game_id = game["id"]
        combined_score = cover_scores.get(game_id, 0) + screenshot_scores.get(game_id, 0)
        
        candidates.append({
            "game_id": game_id,
            "title": game["title"],
            "description": game["description"],
            "critic": game["critic"],
            "platforms": game.get("platforms", []),
            "genres": game.get("genres", []),
            "score": combined_score
        })
    
    # Sort by combined score
    reranked = sorted(candidates, key=lambda x: x["score"], reverse=True)
    
    # Format results
    result = f"Found {len(reranked)} visually similar games:\n\n"
    for i, c in enumerate(reranked[:num_results], 1):
        result += f"{i}. {format_game_result(c, c.get('score'))}\n\n"
    
    return result

# Test the enhanced image search tool (if sample image exists)
try:
    test_result = image_search_tool("../data/sample.jpg", 3)
    print("🧪 Testing enhanced image search tool:")
    print(test_result)
except FileNotFoundError:
    print("⚠️ Sample image not found. Tool ready for use!")
except Exception as e:
    print(f"⚠️ Image search test failed: {e}")
    print("💡 This might be due to ChromaDB HNSW index corruption.")
    print("   The text search works fine, so the issue is with image embeddings.")
    print("   You can continue with text search and fix image search later.")

🖼️ Searching for games similar to: ../data/sample.jpg
🧪 Testing enhanced image search tool:
Found 2 visually similar games:

1. 🎮 Don't Starve + Reign of Giants (Score: -1324.51)
📝 This edition includes:* Don't Starve (base game)

Don't Starve: Reign of Giants (DLC)

For the PlayStation Vita, PS3, Wii U, Xbox One and iOS this is the first and only release of the game. For PC and...
⭐ Critics: I’ve personally enjoyed my time spent in Don’t Starve. The only thing that it’s missing is the new D...
🎯 Platforms: Linux, Windows, Macintosh

2. 🎮 Spirits & Spells (Score: -1446.95)
📝 In Spirits and Spells the player takes the role of the two kids Alicia and Greg which visit a spooky old house with their friends. Unfortunately the Bogeyman steals the soles of their friends and turn...
⭐ Critics: Gleich zwei Charaktere sind in diesem Hüpfspiel spielbar, was aber nicht heißen soll, dass dich auch...
🎯 Platforms: Game Boy Advance




In [7]:
# Tool 3: Database Query Tool for Advanced Filtering
def database_filter_tool(genres: List[str] = None, platforms: List[str] = None, 
                        min_score: float = None, max_score: float = None,
                        release_year: int = None, num_results: int = 10) -> str:
    """
    Filter games using PostgreSQL queries for precise control.
    
    Args:
        genres: List of genres to filter by
        platforms: List of platforms to filter by
        min_score: Minimum MobyGames score
        max_score: Maximum MobyGames score
        release_year: Filter by release year
        num_results: Number of results to return
    
    Returns:
        Formatted string with filtered games
    """
    print(f"🔍 Filtering games with criteria...")
    
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    
    # Build dynamic WHERE clause
    where_conditions = []
    params = []
    
    if genres:
        where_conditions.append("genres && %s")  # Array overlap operator
        params.append(genres)
    
    if platforms:
        where_conditions.append("platforms && %s")
        params.append(platforms)
    
    if min_score is not None:
        where_conditions.append("moby_score >= %s")
        params.append(min_score)
    
    if max_score is not None:
        where_conditions.append("moby_score <= %s")
        params.append(max_score)
    
    if release_year:
        where_conditions.append("EXTRACT(YEAR FROM release_date) = %s")
        params.append(release_year)
    
    where_clause = " AND ".join(where_conditions) if where_conditions else "1=1"
    
    query = f"""
        SELECT id, title, description, release_date, moby_score, moby_url,
               platforms, genres, developers, publishers, cover_path, 
               screenshot_paths, all_critics
        FROM games_with_critics 
        WHERE {where_clause}
        ORDER BY moby_score DESC NULLS LAST
        LIMIT %s
    """
    
    params.append(num_results)
    
    cursor.execute(query, params)
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    
    # Format results
    games = []
    for result in results:
        game_data = dict(result)
        game_data['description'] = clean_html(game_data.get('description', ''))
        
        # Combine critic citations (all_critics is a single string with pipe separators)
        all_critics = game_data.get('all_critics', '')
        if all_critics:
            # Split by pipe and clean each citation
            citations = all_critics.split('|')
            clean_citations = [clean_html(citation.strip()) for citation in citations if citation.strip()]
            game_data['critic'] = ' | '.join(clean_citations)
        else:
            game_data['critic'] = ''
        
        games.append(game_data)
    
    # Build filter description
    filter_desc = []
    if genres:
        filter_desc.append(f"genres: {', '.join(genres)}")
    if platforms:
        filter_desc.append(f"platforms: {', '.join(platforms)}")
    if min_score is not None:
        filter_desc.append(f"min score: {min_score}")
    if max_score is not None:
        filter_desc.append(f"max score: {max_score}")
    if release_year:
        filter_desc.append(f"year: {release_year}")
    
    filter_str = ", ".join(filter_desc) if filter_desc else "all games"
    
    result = f"Found {len(games)} games matching {filter_str}:\n\n"
    for i, game in enumerate(games, 1):
        result += f"{i}. {format_game_result(game, game.get('moby_score'))}\n\n"
    
    return result

# Test the database filter tool
test_result = database_filter_tool(genres=["Action RPG"], min_score=8.0, num_results=3)
print("🧪 Testing database filter tool:")
print(test_result)


🔍 Filtering games with criteria...
🧪 Testing database filter tool:
Found 3 games matching genres: Action RPG, min score: 8.0:

1. 🎮 Elden Ring (Score: 9.30)
⭐ Critics: ELDEN RING is definitively a game worth your dime and your time. A game so massive yet so impressive...
🎯 Platforms: Windows, PlayStation 4, Xbox One

2. 🎮 Mass Effect 2 (Score: 9.10)
📝 Mass Effect 2 is the first game in the series to appear on a PlayStation console and it was released almost a year after the versions for other platforms. The game comes on a single Blu-ray and has ad...
⭐ Critics: Bioware gâte les joueurs PS3 avec cette version de Mass Effect 2. Contenus téléchargeables ajoutés s...
🎯 Platforms: PlayStation 3

3. 🎮 Xenoblade Chronicles (Score: 9.00)
📝 Life in the world evolved on the bodies of two giant motionless world-gods named Bionis and Mechonis. In this world, Shulk, a young man with mysterious origins, comes across a magical sword named Mona...
⭐ Critics: Xenoblade Chronicles donne une leçon à tou

## Create the Enhanced Agent

Now we'll create an agent that can use all three tools for comprehensive game recommendations.

In [8]:
# UI-First Approach: Simplified Agent + Direct Filtering Functions
# This approach separates concerns: Agent handles semantic search, UI handles structured filtering

print("🎯 UI-First Architecture Setup")
print("=" * 50)

# Initialize lightweight LLM for semantic search only
try:
    llm = Ollama(
        model="qwen3:0.6b", 
        request_timeout=30.0,  # Reduced timeout
        temperature=0.1,
        top_p=0.9
    )
    
    # Simplified tools - only semantic search
    tools = [
        FunctionTool.from_defaults(
            fn=text_search_tool,
            name="search_games",
            description="Search for games using natural language descriptions, themes, or gameplay concepts."
        )
    ]
    
    # Only add image search if collections are available
    if cover_collection and screenshot_collection:
        tools.append(
            FunctionTool.from_defaults(
                fn=image_search_tool,
                name="find_similar_games",
                description="Find games visually similar to an uploaded image."
            )
        )
        print("✅ Image search tool added")
    else:
        print("⚠️ Image search skipped (collections unavailable)")
    
    # Set up LlamaIndex settings
    embed_model = HuggingFaceEmbedding(
        model_name="../models/all-MiniLM-L6-v2"
    )
    
    Settings.llm = llm
    Settings.embed_model = embed_model
    
    # Create simplified agent
    agent = ReActAgent(
        tools=tools,
        llm=llm,
        verbose=False,  # Reduced verbosity for memory
        max_iterations=3,  # Reduced iterations
        streaming=False,
        system_prompt="""You are a game recommendation assistant. Use search_games to find games based on user descriptions. 
        Be concise and helpful. Focus on understanding what the user wants to play."""
    )
    
    print(f"✅ Simplified agent created with {len(tools)} tools!")
    for tool in tools:
        print(f"  - {tool.metadata.name}")
        
except Exception as e:
    print(f"❌ Agent creation failed: {e}")
    print("💡 This is likely due to memory constraints. We'll use direct function calls instead.")
    agent = None


🎯 UI-First Architecture Setup
✅ Image search tool added
✅ Simplified agent created with 2 tools!
  - search_games
  - find_similar_games


## UI-Friendly Filtering Functions

These functions are designed to be called directly from a web UI, handling all the complex filtering logic that would be too much for the small LLM.


In [9]:
# UI-Friendly Filtering Functions
# These handle complex filtering logic that would overwhelm the small LLM

def get_filter_options():
    """Get all available filter options for UI dropdowns"""
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Get genres
    cursor.execute("SELECT DISTINCT unnest(genres) as genre FROM games WHERE genres IS NOT NULL ORDER BY genre;")
    genres = [row[0] for row in cursor.fetchall()]
    
    # Get platforms  
    cursor.execute("SELECT DISTINCT unnest(platforms) as platform FROM games WHERE platforms IS NOT NULL ORDER BY platform;")
    platforms = [row[0] for row in cursor.fetchall()]
    
    # Get score range
    cursor.execute("SELECT MIN(moby_score), MAX(moby_score) FROM games WHERE moby_score IS NOT NULL;")
    min_score, max_score = cursor.fetchone()
    
    # Get year range
    cursor.execute("SELECT MIN(EXTRACT(YEAR FROM release_date)), MAX(EXTRACT(YEAR FROM release_date)) FROM games WHERE release_date IS NOT NULL;")
    min_year, max_year = cursor.fetchone()
    
    cursor.close()
    conn.close()
    
    return {
        "genres": genres,
        "platforms": platforms,
        "score_range": {"min": float(min_score), "max": float(max_score)},
        "year_range": {"min": int(min_year), "max": int(max_year)}
    }

def filter_games_ui(genres=None, platforms=None, min_score=None, max_score=None, 
                   min_year=None, max_year=None, num_results=20, offset=0):
    """UI-friendly game filtering with pagination"""
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    
    # Build dynamic WHERE clause
    where_conditions = []
    params = []
    
    if genres:
        where_conditions.append("genres && %s")
        params.append(genres)
    
    if platforms:
        where_conditions.append("platforms && %s")
        params.append(platforms)
    
    if min_score is not None:
        where_conditions.append("moby_score >= %s")
        params.append(min_score)
    
    if max_score is not None:
        where_conditions.append("moby_score <= %s")
        params.append(max_score)
    
    if min_year is not None:
        where_conditions.append("EXTRACT(YEAR FROM release_date) >= %s")
        params.append(min_year)
    
    if max_year is not None:
        where_conditions.append("EXTRACT(YEAR FROM release_date) <= %s")
        params.append(max_year)
    
    where_clause = " AND ".join(where_conditions) if where_conditions else "1=1"
    
    # Get total count for pagination
    count_query = f"SELECT COUNT(*) FROM games_with_critics WHERE {where_clause}"
    cursor.execute(count_query, params)
    total_count = cursor.fetchone()['count']
    
    # Get paginated results
    query = f"""
        SELECT id, title, description, release_date, moby_score, moby_url,
               platforms, genres, developers, publishers, cover_path, 
               screenshot_paths, all_critics
        FROM games_with_critics 
        WHERE {where_clause}
        ORDER BY moby_score DESC NULLS LAST
        LIMIT %s OFFSET %s
    """
    
    params.extend([num_results, offset])
    cursor.execute(query, params)
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    
    # Format results for UI
    games = []
    for result in results:
        game_data = dict(result)
        game_data['description'] = clean_html(game_data.get('description', ''))
        
        # Process critics
        all_critics = game_data.get('all_critics', '')
        if all_critics:
            citations = all_critics.split('|')
            clean_citations = [clean_html(citation.strip()) for citation in citations if citation.strip()]
            game_data['critic'] = ' | '.join(clean_citations[:2])  # Limit for UI
        else:
            game_data['critic'] = ''
        
        games.append(game_data)
    
    return {
        "games": games,
        "total_count": total_count,
        "page_size": num_results,
        "offset": offset,
        "has_more": (offset + num_results) < total_count
    }

def search_games_ui(query, num_results=10):
    """UI-friendly semantic search wrapper"""
    return text_search_tool(query, num_results)

def get_game_details(game_id):
    """Get detailed information for a specific game"""
    return get_game_from_db(game_id)

# Test the UI functions
print("🧪 Testing UI-friendly functions:")
print("=" * 40)

# Test filter options
options = get_filter_options()
print(f"📋 Available genres: {len(options['genres'])} (sample: {options['genres'][:5]})")
print(f"📋 Available platforms: {len(options['platforms'])} (sample: {options['platforms'][:5]})")
print(f"📊 Score range: {options['score_range']['min']:.1f} - {options['score_range']['max']:.1f}")
print(f"📅 Year range: {options['year_range']['min']} - {options['year_range']['max']}")

# Test filtering
filtered = filter_games_ui(genres=["Action RPG"], min_score=8.0, num_results=3)
print(f"🎮 Filtered games: {len(filtered['games'])} results (total: {filtered['total_count']})")
for game in filtered['games']:
    print(f"  - {game['title']} (Score: {game['moby_score']})")

print("✅ UI functions ready!")


🧪 Testing UI-friendly functions:
📋 Available genres: 227 (sample: ['1st-person', '2D scrolling', '3rd-person (Other)', '4X', 'Action'])
📋 Available platforms: 332 (sample: ['1292 Advanced Programmable Video System', '3DO', 'ABC 80', 'Acorn 32-bit', 'Adventure Vision'])
📊 Score range: 2.2 - 9.4
📅 Year range: 1948 - 2029
🎮 Filtered games: 3 results (total: 139)
  - Elden Ring (Score: 9.3)
  - Mass Effect 2 (Score: 9.1)
  - Xenoblade Chronicles (Score: 9.0)
✅ UI functions ready!


## Test the UI-First Approach

Let's test both the simplified agent and the direct UI functions to demonstrate the new architecture.


In [11]:
# Test the UI-First Approach
print("🎮 Testing UI-First Approach")
print("=" * 50)

# Test 1: Direct semantic search (what the agent would do)
print("\n🔍 Test 1: Semantic Search")
print("-" * 30)
try:
    search_result = search_games_ui("survival games with crafting", 3)
    print(search_result)
except Exception as e:
    print(f"❌ Search failed: {e}")

# Test 2: UI filtering (what the frontend would do)
print("\n🎯 Test 2: UI Filtering")
print("-" * 30)
try:
    # Simulate user selecting filters in UI
    filtered_results = filter_games_ui(
        genres=["Action RPG", "Adventure"],
        min_score=8.0,
        min_year=2015,
        num_results=5
    )
    print(f"Found {filtered_results['total_count']} games matching criteria:")
    for game in filtered_results['games']:
        print(f"  - {game['title']} ({game['moby_score']}) - {game['release_date']}")
except Exception as e:
    print(f"❌ Filtering failed: {e}")

# Test 3: Agent interaction (if available)
print("\n🤖 Test 3: Agent Interaction")
print("-" * 30)
if agent:
    try:
        import asyncio
        import nest_asyncio
        nest_asyncio.apply()
        
        response = asyncio.run(agent.run("Find me some good furry games"))
        print("Agent response:")
        print(str(response))
    except Exception as e:
        print(f"❌ Agent failed: {e}")
        print("💡 Using direct search instead:")
        search_result = search_games_ui("furry games", 3)
        print(search_result)
else:
    print("⚠️ Agent not available (memory constraints)")
    print("💡 Using direct search instead:")
    search_result = search_games_ui("furry games", 3)
    print(search_result)

print("\n" + "="*50)


🎮 Testing UI-First Approach

🔍 Test 1: Semantic Search
------------------------------
🔍 Searching for: 'survival games with crafting'
Found 2 games matching 'survival games with crafting':

1. 🎮 Craft the World (Score: 0.79)
📝 Craft The World is a 2D sandbox strategy game, combining the gameplay elements of Terraria and Dwarf Fortress into its own fantasy world, where a group of dwarves attempts to survive against onslaught...
⭐ Critics: Craft the World is a solid game.  Resources are plentiful and creatures are sporadic enough to reall...
🎯 Platforms: Windows, Macintosh, iPhone

2. 🎮 Craft (Score: 0.57)
📝 Craft is a puzzle game where the player is presented with a number of tiles where each piece has a part of a circuit. The objective is to create a complete circuit from an entry point to an exit so th...
🎯 Platforms: SAM Coupé



🎯 Test 2: UI Filtering
------------------------------
Found 197 games matching criteria:
  - Elden Ring (9.3) - 2022-02-25
  - The Witcher 3: Wild Hunt (8.9

## Memory Optimization Summary

Let's demonstrate how we've optimized memory usage for your 8GB RTX 4060 laptop.


In [12]:
import time

print("📊 Performance Comparison: PostgreSQL vs JSONL")
print("=" * 50)

# Test PostgreSQL performance
start_time = time.time()
games_postgres = get_games_from_db([1, 2, 3, 4, 5])
postgres_time = time.time() - start_time

print(f"✅ PostgreSQL: Retrieved {len(games_postgres)} games in {postgres_time:.4f} seconds")
print(f"   Sample game: {games_postgres[0]['title'] if games_postgres else 'None'}")

# Test database filter performance
start_time = time.time()
filtered_games = database_filter_tool(genres=["RPG"], min_score=8.0, num_results=10)
filter_time = time.time() - start_time

print(f"✅ Database Filter: Found filtered games in {filter_time:.4f} seconds")

# Test text search performance
start_time = time.time()
search_results = text_search_tool("fantasy RPG games", num_results=5)
search_time = time.time() - start_time

print(f"✅ Text Search: Found semantic matches in {search_time:.4f} seconds")

print(f"\n🎯 Key Benefits:")
print(f"  - PostgreSQL: Fast metadata queries with proper indexing")
print(f"  - ChromaDB: Efficient vector similarity search")
print(f"  - Structured Data: Proper relationships and data integrity")
print(f"  - Scalability: Can handle millions of records efficiently")
print(f"  - Flexibility: Combine SQL queries with vector search")


📊 Performance Comparison: PostgreSQL vs JSONL
✅ PostgreSQL: Retrieved 5 games in 0.1537 seconds
   Sample game: The X-Files Game
🔍 Filtering games with criteria...
✅ Database Filter: Found filtered games in 0.0483 seconds
🔍 Searching for: 'fantasy RPG games'
✅ Text Search: Found semantic matches in 0.4208 seconds

🎯 Key Benefits:
  - PostgreSQL: Fast metadata queries with proper indexing
  - ChromaDB: Efficient vector similarity search
  - Structured Data: Proper relationships and data integrity
  - Scalability: Can handle millions of records efficiently
  - Flexibility: Combine SQL queries with vector search


## Summary

### What We've Built:
✅ **UI-First Architecture**: Separates concerns between agent and UI  
✅ **Hybrid Database**: PostgreSQL for metadata + ChromaDB for vectors  
✅ **Memory-Optimized**: Simplified agent to avoid memory issues  
✅ **UI-Friendly Functions**: Direct filtering without LLM complexity  
✅ **Production Ready**: Scalable design with proper error handling  

### Key Architecture Decisions:
- **Agent**: Handles semantic search only (lightweight)
- **UI Functions**: Handle complex filtering logic (fast, reliable)
- **PostgreSQL**: Fast metadata queries with proper indexing
- **ChromaDB**: Efficient vector similarity search
- **Memory Management**: Graceful degradation when resources are limited

### Benefits of UI-First Approach:
- **Better UX**: Users see actual filter options, not guess parameter names
- **Faster**: Direct SQL queries instead of LLM reasoning about filters
- **More Reliable**: No complex parameter parsing that can fail
- **Memory Efficient**: Small LLM only handles semantic search
- **Scalable**: UI can handle millions of filter combinations instantly

### Next Steps:
1. **Build Web UI**: Create React/Vue frontend with filter components
2. **API Endpoints**: Flask/FastAPI routes for all UI functions
3. **Caching Layer**: Redis for frequently accessed filter options
4. **Image Search Fix**: Rebuild ChromaDB image collections when needed
5. **Analytics**: Track user preferences and popular searches

🎉 **Perfect!** You now have a memory-efficient, UI-ready Agentic RAG system!
